In [1]:
import json
import requests
import datetime
import pandas as pd
import urllib.request
import codecs
import numpy as np
from six.moves import urllib

In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:95% !important; }</style>"))

In [3]:
#eligibility criteria
volume_cutoff = 0.1
percent_float_cutoff = 0.1

#month of rebalance
old_month = 'may'
new_month = 'June'

In [4]:
#change call to new filepath
asset_db = pd.read_excel('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/utils/rebalance_tools/csvs\\Bletchley Rebalance '+new_month+' 2020.xlsx',\
                         sheetname='assets',index_col='Ticker')

In [5]:
#grab all assets from Messari
#deprecated as they have too many!
#url = "https://data.messari.io/api/v1/assets?with-profiles"
#response = (urllib.request.urlopen(url).read())
#response = response.decode('utf-8')
#d = json.loads(response)
#df = pd.DataFrame(d['data'])
#df = df.set_index((df['symbol']))

In [6]:
df = asset_db
#ticker mapping issues between Bletchley DB and OnChainFX
df.index = map(lambda x: x.lower(), df.index)

In [7]:
url_messari = "https://data.messari.io/api/v1/assets/btc/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)
df_all = pd.Series()

In [8]:
#get Messari data given a ticker
#don't actually use the category or sector data
def get_messari_data(ticker):
    url_messari = "https://data.messari.io/api/v1/assets/{}/metrics".format(ticker)
    response_messari = (urllib.request.urlopen(url_messari).read())
    response_messari = response_messari.decode('utf-8')
    d_messari = json.loads(response_messari)
    df_all = pd.Series()
    if d_messari['data']['supply']['y_2050'] == None:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['circulating'])
    else:
        df_all['Y2050 Supply'] = (d_messari['data']['supply']['y_2050'])
    df_all['Supply % Issued'] = df_all['Y2050 Supply'] / ((df_all['Y2050 Supply']))
    df_all['Price'] = (d_messari['data']['market_data']['price_usd'])
    #df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 30)
    df_all['Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    df_all['Current Marketcap'] = df_all['Price'] * df_all['Y2050 Supply']
    try:
        df_all['Est Volume'] = ((d_messari['data']['market_data']['volume_last_24_hours']) * 180)
    except:
        pass
    return df_all

In [9]:
url_messari = "https://data.messari.io/api/v1/assets/elf/metrics"
response_messari = (urllib.request.urlopen(url_messari).read())
response_messari = response_messari.decode('utf-8')
d_messari = json.loads(response_messari)

In [10]:
#master function to get all Messari data and put in dictionary
metrics_log = dict.fromkeys(df.index)
for t in df.index:
    try:
        metrics = get_messari_data(t)
        metrics_log[t] = metrics
        df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Available Supply')] = metrics['Y2050 Supply']
        df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
        df.loc[t,('Price')] = metrics['Price']
        df.loc[t,('Marketcap')] = metrics['Marketcap']
        df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
        try:
            df.loc[t,('Est Volume')] = metrics['Est Volume']
        except:
            pass
    except:
        pass

In [11]:
metrics_log['elf']

Y2050 Supply         5.444802e+08
Supply % Issued      1.000000e+00
Price                8.916041e-02
Marketcap            4.854608e+07
Current Marketcap    4.854608e+07
Est Volume           4.586225e+09
dtype: float64

In [12]:
#t = 'IOTA'
#metrics = get_messari_data(t)
#t = 'IOT'
#print (metrics)
#metrics_log[t] = metrics
#df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
#df.loc[t,('Available Supply')] = metrics['Available Supply']
#df.loc[t,('Est Volume')] = metrics['Est Volume']
#df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
#df.loc[t,('Price')] = metrics['Price']
#df.loc[t,('Marketcap')] = metrics['Marketcap']
#df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
#df.loc[t,('Category')] = metrics['Category']
#df.loc[t,('Sectors')] = metrics['Sectors']
#try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
#except:
#    pass
#try:
#    df.loc[t,('Category2')] = metrics['Category2']
#except:
#    pass
#try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
#except:
#    pass

In [13]:
# t = 'ELF'
# metrics = get_messari_data(t)
# t = 'ELF_TEST'
# #print (metrics)
# metrics_log[t] = metrics
# metrics['Y2050 Supply']
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#    df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#    pass
# try:
#    df.loc[t,('Category2')] = metrics['Category2']
# except:
#    pass
# try:
#    df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [14]:
# t = 'VET'
# metrics = get_messari_data(t)
# metrics_log[t] = metrics
# df.loc[t,('Y2050 Supply')] = metrics['Y2050 Supply']
# df.loc[t,('Available Supply')] = metrics['Available Supply']
# #df.loc[t,('Est Volume')] = metrics['Est Volume']
# df.loc[t,('Supply % Issued')] = metrics['Supply % Issued']
# df.loc[t,('Price')] = metrics['Price']
# df.loc[t,('Marketcap')] = metrics['Marketcap']
# df.loc[t,('Current Marketcap')] = metrics['Current Marketcap']
# df.loc[t,('Category')] = metrics['Category']
# df.loc[t,('Sectors')] = metrics['Sectors']
# try:
#     df.loc[t,('Est Volume')] = metrics['Est Volume']
# except:
#     pass
# try:
#     df.loc[t,('Category2')] = metrics['Category2']
# except:
#     pass
# try:
#     df.loc[t,('Sectors2')] = metrics['Sectors2']
# except:
#    pass

In [15]:
#change Messari data to uppercase to match other sources
df.index = map(lambda x: x.upper(), df.index)

In [16]:
#df.loc['IOT'] = df.loc['IOTA']
#df.loc['VET'] = df.loc['VEN']

In [17]:
#read in other data sources
onchainfx_db = df

In [18]:
#remove duplicates
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))
#coinmarketcap_db = coinmarketcap_db[~coinmarketcap_db.index.duplicated(keep='first')]
onchainfx_db = onchainfx_db[~onchainfx_db.index.duplicated(keep='first')]
asset_db = asset_db[~asset_db.index.duplicated(keep='first')]
#print (len(coinmarketcap_db))
print (len(onchainfx_db))
print (len(asset_db))

145
145
145
145


In [19]:
#fix any ticker mapping issues - rank date use these
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
#onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
onchainfx_db = onchainfx_db.rename({'IOTA' : 'MIOTA'})
onchainfx_db = onchainfx_db.rename({'IOT' : 'MIOTA'})
asset_db = asset_db.rename({'IOTA' : 'MIOTA'})

In [20]:
# #fix any ticker mapping issues - rebalance day use these
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'HSR' : 'HC'})
# onchainfx_db = onchainfx_db.rename({'IOTA' : 'IOT'})
# asset_db = asset_db.rename({'IOTA' : 'IOT'})

In [21]:
#onchainfx_db['Marketcap'] = np.where(onchainfx_db['Marketcap'].isnull() == True,onchainfx_db['Current Marketcap'],onchainfx_db['Marketcap'])

In [22]:
onchainfx_db.loc['GNO']

Coin                      Gnosis
Class                          3
Sector                       NaN
Quality Exchange?              1
ERC20                          1
ICO ok?                        1
Eligible                       1
Y2050 Supply         1.10459e+06
Available Supply     1.10459e+06
Supply % Issued                1
Price                    24.8925
Marketcap             2.7496e+07
Current Marketcap     2.7496e+07
Est Volume           1.63165e+07
Name: GNO, dtype: object

In [23]:
#read in previous month Bletchley Data
list_indexes = ['10' , '20' , '40' , 'Total' , 'ETH' , 'Currency' , 'Platform' , 'Application']
indexes_old = dict.fromkeys(list_indexes)
indexes_new_tickers = dict.fromkeys(list_indexes)
indexes_new_file = dict.fromkeys(list_indexes)
for k in indexes_old:
    if (k == 'Currency' or k == 'Platform' or k == 'Application'):
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Adj Float'])
    else:
        indexes_old[k] = pd.DataFrame()
        indexes_new_tickers[k] = pd.DataFrame()
        indexes_new_file[k] = pd.DataFrame(columns=['Float'])
        indexes_old[k] = pd.read_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_'+old_month+'_rebal.csv' , index_col=[0] , names=[k,'Float' , 'Adj Float'])

In [24]:
#main function to create table of all assets
def create_universe(onchainfx,assets):
    eligibility = pd.DataFrame(columns = ['Marketcap'])
    eligibility['Marketcap'] = onchainfx['Marketcap']
    eligibility['Current Marketcap'] = onchainfx['Current Marketcap']
    eligibility['Float Percent'] = onchainfx['Supply % Issued']
    eligibility['Float'] = onchainfx['Available Supply']
    eligibility['Price'] = onchainfx['Price']
    eligibility['Total Supply'] = onchainfx['Y2050 Supply']
    eligibility['Volume'] = onchainfx['Est Volume']
    eligibility['Float Pass'] = np.where(eligibility['Float Percent']>=0.1, 1, 0)
    eligibility['Volume Ratio'] = eligibility['Volume'] / eligibility['Current Marketcap']
    eligibility['Volume Pass'] = np.where(eligibility['Volume Ratio']>=volume_cutoff, 1, 0)
    for t in eligibility.index:
        if t in assets.index:
            eligibility.loc[t,('Other Pass')] = assets.loc[t,('Eligible')]
            eligibility.loc[t,('Class')] = assets.loc[t,('Class')]
            eligibility.loc[t,('ERC20')] = assets.loc[t,('ERC20')]
        else:
            print (t,' no other data!')
            eligibility.loc[t,('Other Pass')] = 0
    eligibility['Score'] = eligibility['Volume Pass'] + eligibility['Other Pass'] + eligibility['Float Pass']
    eligibility['Eligible'] = np.where(eligibility['Score']>=2.99999, 1, 0)
    #check if duplicates
    count = len(eligibility.index)
    count1 = len(eligibility.index.unique())
    if count != count1:
        print ('duplicate tickers')
    return eligibility

In [25]:
#rank assets by 2050 marketcap for selection list
def rank_and_sort(df):
    df['Rank'] = df['Marketcap'].rank(ascending=False).astype(int)
    df_sorted = df.sort_values('Rank')
    return df_sorted

In [26]:
#get list of eligible assets and rank them
#really noisy error logging right now due to Messari having 4000 assets
eligibility_db = create_universe(onchainfx_db,asset_db)
eligible_db = eligibility_db[eligibility_db.Eligible != 0]
ineligible_db = eligibility_db[eligibility_db.Eligible == 0]

In [27]:
#make sure I've correctly excluded all of these
double_check = ineligible_db[ineligible_db['Volume Pass'] == 1]
double_check

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible
CENNZ,6.519829e+07,6.519829e+07,1.0,8.511038e+08,0.076604,8.511038e+08,2.307320e+07,1,0.353893,1,0.0,3.0,0.0,2.0,0
CTXC,2.710936e+07,2.710936e+07,1.0,2.539879e+08,0.106735,2.539879e+08,2.033428e+09,1,75.008342,1,0.0,2.0,0.0,2.0,0
DENT,2.004785e+07,2.004785e+07,1.0,1.000000e+11,0.000200,1.000000e+11,8.531078e+07,1,4.255357,1,0.0,3.0,0.0,2.0,0
DRGN,2.732338e+07,2.732338e+07,1.0,2.384219e+08,0.114601,2.384219e+08,4.004503e+07,1,1.465596,1,0.0,3.0,0.0,2.0,0
ETN,8.663886e+07,8.663886e+07,1.0,1.011779e+10,0.008563,1.011779e+10,2.233945e+08,1,2.578457,1,0.0,3.0,0.0,2.0,0
GRIN,5.387089e+08,5.387089e+08,1.0,9.771011e+08,0.551334,9.771011e+08,2.957162e+09,1,5.489351,1,0.0,1.0,0.0,2.0,0
KCS,7.893838e+07,7.893838e+07,1.0,8.125028e+07,0.971546,8.125028e+07,1.321739e+09,1,16.743929,1,0.0,2.0,0.0,2.0,0
KIN,6.000806e+07,6.000806e+07,1.0,9.995502e+12,0.000006,9.995502e+12,7.203204e+06,1,0.120037,1,0.0,2.0,0.0,2.0,0
MKR,4.506496e+08,4.506496e+08,1.0,9.966470e+05,452.165682,9.966470e+05,1.809517e+09,1,4.015353,1,0.0,3.0,0.0,2.0,0
PLR,2.066121e+07,2.066121e+07,1.0,8.000000e+08,0.025827,8.000000e+08,3.754699e+06,1,0.181727,1,0.0,3.0,0.0,2.0,0


In [28]:
#eligible_db = eligible_db.append(ineligible_db.loc['XTZ'])
eligible_db = eligible_db.drop('ATOM')

In [29]:
#create eligible universe - not sure on why there is a warning
universe = rank_and_sort(eligible_db)

In [30]:
universe = universe.rename({'HSR' : 'HC'})
universe = universe.rename({'IOT' : 'MIOTA'})
universe = universe.rename({'VEN' : 'VET'})
universe

,Marketcap,Current Marketcap,Float Percent,Float,Price,Total Supply,Volume,Float Pass,Volume Ratio,Volume Pass,Other Pass,Class,ERC20,Score,Eligible,Rank
BTC,2.001240e+11,2.001240e+11,1.0,2.098634e+07,9535.920748,2.098634e+07,5.150119e+12,1,25.734637,1,1.0,1.0,0.0,3.0,1,1
ETH,3.212240e+10,3.212240e+10,1.0,1.351356e+08,237.704873,1.351356e+08,2.365691e+12,1,73.646162,1,1.0,2.0,0.0,3.0,1,2
XRP,2.044295e+10,2.044295e+10,1.0,1.000000e+11,0.204430,1.000000e+11,2.824689e+11,1,13.817423,1,1.0,2.0,0.0,3.0,1,3
XTZ,5.603908e+09,5.603908e+09,1.0,1.923801e+09,2.912936,1.923801e+09,2.126050e+10,1,3.793871,1,1.0,2.0,0.0,3.0,1,4
BCH,5.092564e+09,5.092564e+09,1.0,2.098243e+07,242.706072,2.098243e+07,5.920844e+11,1,116.264494,1,1.0,1.0,0.0,3.0,1,5
LINK,4.361247e+09,4.361247e+09,1.0,1.000000e+09,4.361247,1.000000e+09,8.532514e+10,1,19.564394,1,1.0,3.0,1.0,3.0,1,6
BSV,4.058834e+09,4.058834e+09,1.0,2.098214e+07,193.442293,2.098214e+07,2.795727e+11,1,68.880065,1,1.0,1.0,0.0,3.0,1,7
LTC,3.922638e+09,3.922638e+09,1.0,8.388054e+07,46.764579,8.388054e+07,4.900296e+11,1,124.923470,1,1.0,1.0,0.0,3.0,1,8
XLM,3.769973e+09,3.769973e+09,1.0,5.000000e+10,0.075399,5.000000e+10,1.129891e+11,1,29.970800,1,1.0,1.0,0.0,3.0,1,9
EOS,3.685565e+09,3.685565e+09,1.0,1.366412e+09,2.697258,1.366412e+09,3.729972e+11,1,101.204892,1,1.0,2.0,0.0,3.0,1,10


In [31]:
#create ten index membership
ten_auto = universe[universe['Rank'] <= 8]
indexes_new_tickers['10'] = pd.DataFrame()
indexes_new_tickers['10'] = ten_auto
ten_band = universe[universe['Rank'] <= 12]
ten_band = ten_band[ten_band['Rank'] > 8]
count_ten_members = len(ten_auto)
for t in ten_band.index:
    if t in indexes_old['10'].index:
        if count_ten_members < 10:
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
            print ('Retained' , t)
        else:
            print ('10 full! No room for' , t)
for t in ten_band.index:
    if t not in indexes_old['10'].index:
        if count_ten_members < 10:
            print ('New add' , t)
            indexes_new_tickers['10'] = indexes_new_tickers['10'].append(ten_band.loc[t])
            count_ten_members += 1
        else:
            print ('10 full! Do not add ' , t)

Retained XLM
Retained EOS
10 full! No room for ADA
10 full! Do not add  BNB


In [32]:
#create twenty index membership
twenty_auto = universe[universe['Rank'] <= 26]
for t in twenty_auto.index:
    if t in indexes_new_tickers['10'].index:
        twenty_auto = twenty_auto.drop(t)
#remove all rows in indexes_new_tickers['10']
indexes_new_tickers['20'] = pd.DataFrame()
indexes_new_tickers['20'] = twenty_auto
twenty_band = universe[universe['Rank'] <= 34]
twenty_band = twenty_band[twenty_band['Rank'] > 26]
count_twenty_members = len(twenty_auto)
for t in twenty_band.index:
    if t in indexes_old['20'].index:
        if count_twenty_members < 20:
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
            print ('Retained' , t)
        else:
            print ('20 full! No room for' , t)
for t in twenty_band.index:
    if t not in indexes_old['20'].index:
        if count_twenty_members < 20:
            print ('New add' , t)
            indexes_new_tickers['20'] = indexes_new_tickers['20'].append(twenty_band.loc[t])
            count_twenty_members += 1
        else:
            print ('20 full! Do not add ' , t)

Retained DGB
Retained ZRX
Retained BAT
Retained DCR
20 full! No room for LSK
20 full! Do not add  THETA
20 full! Do not add  OMG
20 full! Do not add  SC


In [33]:
#create forty index membership
forty_auto = universe[universe['Rank'] <= 60]
for t in forty_auto.index:
    if t in indexes_new_tickers['10'].index:
        forty_auto = forty_auto.drop(t)
    if t in indexes_new_tickers['20'].index:
        forty_auto = forty_auto.drop(t)
indexes_new_tickers['40'] = pd.DataFrame()
indexes_new_tickers['40'] = forty_auto
forty_band = universe[universe['Rank'] <= 80]
forty_band = forty_band[forty_band['Rank'] > 60]
count_forty_members = len(forty_auto)
for t in forty_band.index:
    if t in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('Retained' , t)
        else:
            print ('40 full! No room for' , t)
for t in forty_band.index:
    if t not in indexes_old['40'].index:
        if count_forty_members < 40:
            indexes_new_tickers['40'] = indexes_new_tickers['40'].append(forty_band.loc[t])
            count_forty_members += 1
            print ('New add' , t)
        else:
            print ('40 full! Do not add ' , t)

Retained MAID
Retained IOST
Retained STORJ
Retained MANA
Retained GXC
Retained GNT
Retained ARDR
Retained ELF
Retained WAN
Retained FUN
40 full! Do not add  LRC
40 full! Do not add  WAX
40 full! Do not add  POLY
40 full! Do not add  BNT
40 full! Do not add  RDD
40 full! Do not add  STRAT
40 full! Do not add  POWR
40 full! Do not add  LBC
40 full! Do not add  ANT
40 full! Do not add  QASH


In [34]:
#construct total index from 10 + 20 + 40
indexes_new_tickers['Total'] = indexes_new_tickers['10'].append(indexes_new_tickers['20']).append(indexes_new_tickers['40'])

In [35]:
#construct various sector indexes from total index
indexes_new_tickers['ETH'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['ERC20'] == 1]
indexes_new_tickers['Currency'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 1]
indexes_new_tickers['Platform'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 2]
indexes_new_tickers['Application'] = indexes_new_tickers['Total'][indexes_new_tickers['Total']['Class'] == 3]

In [36]:
#print out membership changes for checking and blog
for k in indexes_old:
    print (k , 'adds' , indexes_new_tickers[k].index.difference(indexes_old[k].index).values)
    print (k , 'deletes' , indexes_old[k].index.difference(indexes_new_tickers[k].index).values)
    print (k , 'count members:' , len(indexes_new_tickers[k]))
    if indexes_new_tickers[k].index.duplicated(keep='first').any() == True:
        print ('Duplicates if True' , indexes_new_tickers[k].index.duplicated(keep='first'))
    print ("")
sectors = len(indexes_new_tickers['Currency'])+len(indexes_new_tickers['Platform'])+len(indexes_new_tickers['Application'])
print ('Sector count members:' , sectors)

10 adds ['LINK']
10 deletes ['ADA']
10 count members: 10

Application adds ['DGD']
Application deletes ['CVC']
Application count members: 25

Total adds ['DGD']
Total deletes ['CVC']
Total count members: 70

20 adds ['ADA' 'ZIL']
20 deletes ['LINK' 'LSK']
20 count members: 20

Currency adds []
Currency deletes []
Currency count members: 19

ETH adds ['DGD']
ETH deletes ['CVC']
ETH count members: 20

Platform adds []
Platform deletes []
Platform count members: 26

40 adds ['DGD' 'LSK']
40 deletes ['CVC' 'ZIL']
40 count members: 40

Sector count members: 70


In [37]:
#write off new index memberships for use in rebalance
for k in indexes_new_tickers:
    if 'MIOTA' in indexes_new_tickers[k].index:
        indexes_new_tickers[k] = indexes_new_tickers[k].rename({'MIOTA' : 'IOT'})
    indexes_new_tickers[k].to_csv('C:/Users/stoin/Desktop/Mike/Bletch/bcindex/index-data/src/main/resources/business_rules/'+k+'_pre_'+new_month+'.csv',columns=['Float'])